### Imports

In [65]:
from z3 import *
from itertools import combinations
from sat_encodings import *

### Constraint encodings

### Solving

- Teams: $N$
- Periods: $P = N/2$
- Weeks: $W = N - 1$
- Round robin tournament: $rb$, a $P \times W$ matrix of tuples $t = (t_1, t_2)$ where $t_1 < t_2$
- Variables:
    - $index_{p,w,t}$: teams $rb[t][w]$ play in period $p$ of weeek $w$
    - $teams_{t_1, t_2, p}$: $t_1$ plays against $t_2$ in period $p$
- Constraints:
    - Every team plays at most once a week:
        - $\forall p \in P, w \in W.(\forall t \in P. exactly\_one(index_{p,w,t}))$
        - $\forall w \in W, t \in P.(\forall p \in P. exactly\_one(index_{p,w,t}))$
    - Every team plays between once and twice a week
        - $\forall p \in P, w \in W, t \in P.(index_{p,w,t} \leftrightarrow teams_{rb[t][w][0], rb[t][w][1], p})$
        - $\forall p \in P, t_1 \in N.(\forall t_2 \in N. (t_1 < t_2 \rightarrow at\_most\_2(teams_{t_1, t_2, p}))$

In [66]:
def solve_instance(rb, P, W, N):
    index_teams = [[[Bool(f"index_teams{p}_{w}_{t}") for t in range(P)] for w in range(W)] for p in range(P)]
    teams = [[[Bool(f"team{ti}_{tj}_{p}") for p in range(P)] for tj in range(N)] for ti in range(N)]

    solutions = {}

    s_sat = Solver()

    """
        SATISFIABILITY
    """ 
    # every team plays at most once every week
    for p in range(P):
       for w in range(W):
            s_sat.add(exactly_one_he([index_teams[p][w][t] for t in range(P)]))
    for w in range(W):
        for t in range(P):
            s_sat.add(exactly_one_he([index_teams[p][w][t] for p in range(P)]))

    # every team plays between once and twice in each period
    for p in range(P):
        for w in range(W):
            for t in range(P):
                s_sat.add(Iff(index_teams[p][w][t], teams[rb[t][w][0]][rb[t][w][1]][p]))
    for bp in range(P):
        s_sat.add(at_most_k_seq([teams[0][t2][bp] for t2 in range(1, N)], 2))
        s_sat.add(at_most_k_seq([teams[t2][-1][bp] for t2 in range(0, N-1)], 2))
        for t1 in range(1, N-1):
            s_sat.add(at_most_k_seq([teams[t2][t1][bp] for t2 in range(t1)]+[teams[t1][t2][bp] for t2 in range(t1+1, N)], 2))


    s_sat.check()
    model_sat = s_sat.model()


    sat_solution = []
    for p in range(P):
        matches = []
        for w in range(W):
            new_p = [model_sat.evaluate(index_teams[p][w][bp]) for bp in range(P)].index(True)
            matches.append(rb[new_p][w])
        sat_solution.append(matches)

    solutions['sat'] = sat_solution

    """
        OPTIMIZATION
    """
    low = 0
    high = N
    index_roles = [[Bool(f"role_{p}_{w}") for w in range(W)] for p in range(P)]
    roles =       [[Bool(f"role_{ti}_{tj}") for tj in range(N)] for ti in range(N)]
    while low <= high:
        mid = (low + high) // 2

        s_opt = Solver()

        # binding
        for p in range(P):
            for w in range(W):
                s_opt.add(Iff(index_roles[p][w], roles[rb[p][w][0]][rb[p][w][1]]))

        s_opt.add(at_least_k_seq([roles[0][t2] for t2 in range(1, N)], mid))
        s_opt.add(at_least_k_seq([Not(roles[t2][-1]) for t2 in range(0, N-1)], mid))
        for t1 in range(1, N-1):
            s_opt.add(at_least_k_seq([Not(roles[t2][t1]) for t2 in range(t1)] + [roles[t1][t2] for t2 in range(t1 + 1, N)], mid))

        if s_opt.check() == sat:
            model_opt = s_opt.model()
            solution_opt = []
            for p in range(P):
                matches = []
                for w in range(W):
                    new_r = bool(model_opt.evaluate(index_roles[p][w]))
                    matches.append((sat_solution[p][w][new_r], sat_solution[p][w][1 - new_r]))
                solution_opt.append(matches)
            solutions[f'opt_{mid}'] = solution_opt

            low = mid + 1
        else:
            high = mid - 1

    return solutions

In [67]:
N = 6
P = N//2
W = N-1
S = 2

# round robin tournament with ordered slots
rb = []
for p in range(P):
    matches = []
    for w in range(W):
        if p == 0:
            matches.append(tuple(sorted([N-1, w])))
        else:
            matches.append(tuple(sorted([(p+w) % (N-1), (N-p+w-1)%(N-1)])))
    rb.append(matches)

results = solve_instance(rb, P, W, N)

In [68]:
for name, res in results.items():
    print(f"{name}")
    for p in range(P):
        for w in range(W):
            print(f"({res[p][w][0]:<{int(math.log10(N)) + 1}},{res[p][w][1]:<{int(math.log10(N)) + 1}})", end=" ")
        print()

sat
(0,5) (3,4) (0,4) (3,5) (1,2) 
(1,4) (1,5) (2,5) (2,4) (0,3) 
(2,3) (0,2) (1,3) (0,1) (4,5) 
opt_1
(0,5) (4,3) (4,0) (5,3) (2,1) 
(4,1) (5,1) (5,2) (4,2) (3,0) 
(3,2) (0,2) (3,1) (1,0) (5,4) 
opt_2
(5,0) (3,4) (4,0) (3,5) (1,2) 
(1,4) (5,1) (5,2) (4,2) (0,3) 
(2,3) (0,2) (1,3) (0,1) (5,4) 
